# Algoritmo CART en PySpark
## Escuela Superior de Cómputo - Instituto Politécnico Nacional

### Equipo
* Armas Ramírez Daniel
* Porto García Ismael

**Docente**:
Miguel Sáncez Brito

In [58]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import sum, count, avg, collect_set
from typing import List, Tuple, Dict
from pyspark.sql.types import DoubleType, IntegerType
import math

spark = SparkSession.builder.appName("DecisionTree").getOrCreate()

In [67]:
def rotate(df: DataFrame) -> DataFrame:
    """
    Función para rotar el dataframe
    
    Parameters:
        df(DataFrame): Dataframe al cual se le hará transpose    
    Returns:
        DataFrame: Un nuevo dataframe aplicando la rotación
    """
    rows = df.collect()
    row_1 = df.columns
    n = len(row_1)
    n_rows = len(rows)
    data = [ [] for _ in range(n)]
    for i in range(n):
        data[i].append(row_1[i])

    for i in range(n_rows):
        for j in range(n):
            data[j].append(rows[i][j])
    
    new_cols = data[0]
    new_rows = data[1:]
    rotate_df = spark.createDataFrame(new_rows, new_cols)
    
    print(new_cols)
    dcols = input("Selecciona las columnas a convertir en double, separadas por coma: ").strip()
    dcols = [col.strip() for col in dcols.split(",") if col.strip() != ""]

    icols = input("Selecciona las columnas a convertir en int, separadas por coma: ").strip()
    icols = [col.strip() for col in icols.split(",") if col.strip() != ""]
    
    # convertir a flotante
    for col in dcols:
        rotate_df = rotate_df.withColumn(col, rotate_df[col].cast(DoubleType()))

    #convertir a entero
    for col in icols:
        rotate_df = rotate_df.withColumn(col, rotate_df[col].cast(IntegerType()))
    return rotate_df

In [39]:
def transpose(cols: List[str], df: DataFrame, colgroup: str) -> DataFrame:
    """
    Función para transponer el dataframe
    
    Parameters:
        cols(List[str]): Lsita de las columnas del DataFrame
        df(DataFrame): Dataframe al cual se le hará transpose
        colgroup(str): columna usada para agrupar
    
    Returns:
        DataFrame: Una copia del dataframe aplicando Transpose
    """
    rows = []
    columns = df.agg(collect_set(colgroup)).collect()[0][0]
    columns.insert(0, 'temp-col')
    for col in cols:
        if col == colgroup:
            continue
        count_v = df.groupBy(colgroup, col).agg(count("*")).collect()
        for item in count_v:
            temp_row = [f"{col}-{item[col]}"]
            for i in range(1, len(columns)):
                if columns[i] == item[colgroup]:
                    temp_row.append(item['count(1)'])
                else:
                    temp_row.append(0)
            rows.append(temp_row)

    transpose_df = spark.createDataFrame(rows, columns)
    return transpose_df


def read_csv(path:str) -> List[str]:
    """
    Función para leer el archivo csv
    Parameters:
        path(str): path de ubicación del archivo csv
    Returns:
        List[DataFrame, List[str]]: retorna el DataFrame y la lista de las columnas
    """
    
    try:
        df = spark.read.csv(path, inferSchema=True, header=True)
        cols = df.columns

    except Exception as e:
        print(f'Error leyendo el archivo')
        print(str(e))
    else:
        return df, cols

In [68]:
def get_prediction_vect(df: DataFrame) -> Dict[str, str]:
    print("Columnas disponibles en el dataframe:")
    print(df.columns)
    
    target_col = input("Selecciona la columna a predecir (target): ").strip()
    
    feature_cols_input = input("Selecciona las columnas a usar como predictores, separadas por comas: ").strip()
    predictors = [col.strip() for col in feature_cols_input.split(",") if col.strip() != ""]
    
    return predictors, target_col

In [69]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.types import StringType

def prepare_data(df, target_col, predictors):
    data = df

    if isinstance(df.schema[target_col].dataType, StringType):
        si_target = StringIndexer(inputCol=target_col, outputCol="label")
        si_model_target = si_target.fit(data)
        data = si_model_target.transform(data)
        print(f"\nCodificación de '{target_col}':")
        for i, lbl in enumerate(si_model_target.labels):
            print(f"{i} -> {lbl}")
    else:
        data = data.withColumnRenamed(target_col, "label")

    feat_cols = []
    for col_name in predictors:
        if col_name == target_col or col_name.lower() == "id":
            continue

        if isinstance(df.schema[col_name].dataType, StringType):
            si_feat = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_idx")
            si_model_feat = si_feat.fit(data)
            data = si_model_feat.transform(data)
            feat_cols.append(f"{col_name}_idx")
            print(f"\nCodificación de '{col_name}':")
            for i, lbl in enumerate(si_model_feat.labels):
                print(f"{i} -> {lbl}")
        else:
            feat_cols.append(col_name)

    assembler = VectorAssembler(inputCols=feat_cols, outputCol="features")
    data = assembler.transform(data)

    train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

    print("\nMuestra de train_data:")
    train_data.show(5, truncate=False)

    return train_data, test_data


In [70]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import col

def fit_model(train_data, test_data):

    max_depth = float(input("Ingresa max_depth a usar: "))
    min_instances = float(input("Ingresa min_instances a usar: "))
    
    dt = DecisionTreeClassifier(
        labelCol="label",
        featuresCol="features",
        maxDepth=max_depth,
        minInstancesPerNode=min_instances
    )

    # Entrenar
    model = dt.fit(train_data)
    predictions = model.transform(test_data)

    # Accuracy
    correct = predictions.filter(col("prediction") == col("label")).count()
    total = predictions.count()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy}")

    # Falsos positivos por clase
    labels = [row[0] for row in predictions.select("label").distinct().collect()]
    print("Falsos Positivos por clase:")
    for l in labels:
        fp = predictions.filter((col("prediction") == l) & (col("label") != l)).count()
        print(f"Clase {l}: {fp}")

    # arbol
    print("\nÁrbol de decisión:")
    print(model.toDebugString)

    return model, predictions


In [71]:
from pyspark.ml.linalg import Vectors

def predecir(model):
    while True:
        input_str = input("Ingresa las características separadas por comas (ej: 0.0,0.2,3.5): ")
        if input_str == "":
            break
        
        features = [float(x.strip()) for x in input_str.split(",")]
    
        df = spark.createDataFrame([(Vectors.dense(features),)], ["features"])
    
        prediction = model.transform(df).collect()[0]["prediction"]
        print(f"Predicción del modelo: {prediction}")


In [73]:
print("Bienvenido al uso del modelo Decision Tree")
path = input("Por favor provee el path de ubicación del archivo csv: ")
df, cols = read_csv(path)

print("\n\n")
print("Muestra del Dataframe")
df.show(5)

rotate_in = input("Desea rotar el Dataframe: ")
if rotate_in == 'yes':
    df = rotate(df)
    print("\n\n")
    print("Muestra del Dataframe")
    df.show(5)

predictors, target_col = get_prediction_vect(df)
train_data, test_data = prepare_data(df, target_col, predictors)

model, predictions = fit_model(train_data, test_data)
predecir(model)

Bienvenido al uso del modelo Decision Tree


Por favor provee el path de ubicación del archivo csv:  temp.csv





Muestra del Dataframe
+-----------+-----------+-----------+-----------+-----------+-----------+
|         Id|          1|          2|          3|          4|          5|
+-----------+-----------+-----------+-----------+-----------+-----------+
|SepalLength|        5.1|        4.9|        4.7|        4.6|        5.0|
| SepalWidth|        3.5|        3.0|        3.2|        3.1|        3.6|
|PetalLength|        1.4|        1.4|        1.3|        1.5|        1.4|
| PetalWidth|        0.2|        0.2|        0.2|        0.2|        0.2|
|    Species|Iris-setosa|Iris-setosa|Iris-setosa|Iris-setosa|Iris-setosa|
+-----------+-----------+-----------+-----------+-----------+-----------+



Desea rotar el Dataframe:  yes


['Id', 'SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']


...

Ingresa max_depth a usar:  2
Ingresa min_instances a usar:  3


Test Accuracy: 1.0
Falsos Positivos por clase:
Clase 0.0: 0

Árbol de decisión:
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_1b508bc709a4, depth=0, numNodes=1, numClasses=1, numFeatures=4
  Predict: 0.0



Ingresa las características separadas por comas (ej: 0.0,0.2,3.5):  4.0, 3.2, 2.4, 4.5


Predicción del modelo: 0.0


Ingresa las características separadas por comas (ej: 0.0,0.2,3.5):  
